# Commerical Satellite Imagery Tasking via the CSSP Imagery Derived Requirements Tool
Builds the `tasking` table from a locally stored Excel spreadsheet that should be created from a Google Drive Sheet created by Meredith Sackett.

### Import libraries

In [1]:
# Basic stack
import os
import shutil
from glob import glob
import warnings

# Web stack
import json

# Database stack
import sqlite3

# Data Science stack
import numpy as np
import pyproj
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import utm
import pandas as pd
import geopandas as gpd
import shapely
from shapely import to_geojson
from shapely.ops import transform
from shapely.geometry import Polygon
from fiona.drvsupport import supported_drivers
import folium

### Environmental settings

In [2]:
warnings.filterwarnings("ignore")

### User defined variables

In [3]:
db = "../../db.sqlite3"
dar_spreadsheet = "../../data/databases/Tracking - CIDR Requests - live whales.xlsx"

### User defined functions

In [4]:
def insert_tasking(c, dar_id, aoi_id):
    """ Inserts CIDR DARs into the DARS Data Table. The DAR
            is the Primary Key for the datatable since it is
            unique.

        C - A cursor
        DAR ID - An Entity ID value from EarthExplorer
    """
    sql_string = f"INSERT INTO tasking(dar, aoi) VALUES (\"{dar_id}\", \"{aoi_id}\")"
    c.execute(sql_string)

### Read in Excel spreadsheet, show the head

In [5]:
df = pd.read_excel(dar_spreadsheet, sheet_name = 'Master')
print("Your initial DAR spreadsheet is {} in size".format(df.shape))

df.head(2)

Your initial DAR spreadsheet is (220, 37) in size


,DAR,AOI,Target Identified in Imagery,Species,Submitted by:,Date Entered,Tasking Description,Acquisition Start,Unnamed: 8,Acquisition End,...,Shared with Ludwig,Shared with Olivia Pisano,ArcPro Data Maps Saved in Drive,Reprocessing,Comments,Location,Website link,Permission to Share,Wind Data Available from National Buoy Data Center (download link to station buoy ID archives),Manual Annotation
0,6826.0,69,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,Entrance to Chesapeake Bay,2023-11-29,-,2024-11-29,...,False,False,False,NaN,NaN,NaN,https://cidr.cr.usgs.gov/request/view/6826/,Imagery will be shared with GAIA project colla...,NaN,NaN
1,6827.0,70,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,Entrance to New York Harbor,2023-11-29,-,2024-11-29,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop table
This is for demonstration and troubleshooting purposes.

In [6]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS tasking''')
conn.commit()
conn.close()

### Create `tasking` table

In [7]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS tasking(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        dar INTEGER,
        aoi INTEGER,
        location VARCHAR(12),
        target VARCHAR(30),
        requestor VARCHAR(16),
        vendor VARCHAR(10),            -- Missing from Tracking?
        mono_stereo VARCHAR(6),        -- Missing from Tracking?
        date_entered DATE,
        acquisition_start DATE,
        acquisition_end DATE,
        ona_wv2 VARCHAR(10),
        ona_wv3 VARCHAR(10),
        tasking_description VARCHAR(100),
        comments VARCHAR(250),
        status VARCHAR(8),
        output_format VARCHAR(7),
        processing_level VARCHAR(41),
        website_link VARCHAR(50),
        permission_to_share VARCHAR(500),
        FOREIGN KEY (aoi)
            REFERENCES aoi(id),
        FOREIGN KEY (target)
            REFERENCES targets(target),
        FOREIGN KEY (requestor)
            REFERENCES people(name)
    )
''')

conn.commit()
conn.close()

### Filter to only columns that will be in database

In [8]:
keep_columns = ['DAR', 'AOI', 'Location', 'Species', 'Submitted by:', 'Date Entered',
                'Acquisition Start', 'Acquisition End', 'ONA Restriction - WV2',
                'ONA Restriction - WV3', 'Tasking Description', 'Comments', 'Status of Tasking',
                'Output Format', 'Processing Level', 'Website link', 'Permission to Share']
df = df[keep_columns]
df.head(2)

,DAR,AOI,Location,Species,Submitted by:,Date Entered,Acquisition Start,Acquisition End,ONA Restriction - WV2,ONA Restriction - WV3,Tasking Description,Comments,Status of Tasking,Output Format,Processing Level,Website link,Permission to Share
0,6826.0,69,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,2023-11-29,2024-11-29,NaN,NaN,Entrance to Chesapeake Bay,NaN,In progress,GeoTiff,Radiometrically Corrected (1B),https://cidr.cr.usgs.gov/request/view/6826/,Imagery will be shared with GAIA project colla...
1,6827.0,70,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,2023-11-29,2024-11-29,NaN,NaN,Entrance to New York Harbor,NaN,In progress,GeoTiff,Radiometrically Corrected (1B),NaN,NaN


### Rename columns to those in database

In [9]:
rename_columns = {'DAR': 'dar', 'AOI': 'aoi', 'Location': 'location', 'Species': 'target',
                  'Submitted by:': 'requestor', 'Date Entered': 'date_entered',
                  'Acquisition Start': 'acquisition_start', 'Acquisition End': 'acquisition_end',
                  'ONA Restriction - WV2': 'ona_wv2', 'ONA Restriction - WV3': 'ona_wv3',
                  'Tasking Description': 'tasking_description', 'Comments': 'comments',
                  'Status of Tasking': 'status', 'Output Format': 'output_format',
                  'Processing Level': 'processing_level', 'Website link': 'website_link',
                  'Permission to Share': 'permission_to_share'}
df = df.rename(columns=rename_columns)
df.head(2)

,dar,aoi,location,target,requestor,date_entered,acquisition_start,acquisition_end,ona_wv2,ona_wv3,tasking_description,comments,status,output_format,processing_level,website_link,permission_to_share
0,6826.0,69,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,2023-11-29,2024-11-29,NaN,NaN,Entrance to Chesapeake Bay,NaN,In progress,GeoTiff,Radiometrically Corrected (1B),https://cidr.cr.usgs.gov/request/view/6826/,Imagery will be shared with GAIA project colla...
1,6827.0,70,NaN,North Atlantic right whale,Meredith Sackett,2023-11-29,2023-11-29,2024-11-29,NaN,NaN,Entrance to New York Harbor,NaN,In progress,GeoTiff,Radiometrically Corrected (1B),NaN,NaN


### Insert values from spreadsheet into `tasking` table
Note that some of the DAR values are `NaN`.

In [10]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in df.iterrows():
    try:
        dar = row['dar']
        aoi = row['aoi']
        location = row['location']
        target = row['target']
        requestor = row['requestor']
        date_entered = row['date_entered']
        acquisition_start = row['acquisition_start']
        acquisition_end = row['acquisition_end']
        ona_wv2 = row['ona_wv2']
        ona_wv3 = row['ona_wv3']
        tasking_description = row['tasking_description']
        comments = row['comments']
        status = row['status']
        output_format = row['output_format']
        processing_level = row['processing_level']
        website_link = row['website_link']
        permission_to_share = row['permission_to_share']
        try:
            sql_string = f"INSERT INTO tasking(dar, aoi, location, target, requestor," +\
                            f"date_entered, acquisition_start, acquisition_end, ona_wv2," +\
                            f"ona_wv3, tasking_description, comments, status, output_format," +\
                            f"processing_level, website_link, permission_to_share)" +\
                            f"VALUES (\"{dar}\", \"{aoi}\", \"{location}\", \"{target}\"," +\
                                f"\"{requestor}\", \"{date_entered}\", \"{acquisition_start}\"," +\
                                f"\"{acquisition_end}\",\"{ona_wv2}\",\"{ona_wv3}\",\"{tasking_description}\"," +\
                                f"\"{comments}\",\"{status}\",\"{output_format}\",\"{processing_level}\"," +\
                                f"\"{website_link}\",\"{permission_to_share}\")"
            c.execute(sql_string)
        except Exception as e:
            print("\tException: {} was raised for DAR ID {}".format(e, dar))
    except Exception as e:
        print("Exception: {} was raised for iteration {}".format(e, i))
print("Done updating DARS table!")

conn.commit()
conn.close()

Done updating DARS table!


### Select new information, review it for validation

In [11]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

df = pd.read_sql_query(f"SELECT * FROM tasking", conn)

conn.commit()
conn.close()

df.head()

,id,dar,aoi,location,target,requestor,vendor,mono_stereo,date_entered,acquisition_start,acquisition_end,ona_wv2,ona_wv3,tasking_description,comments,status,output_format,processing_level,website_link,permission_to_share
0,1,6826,69,nan,North Atlantic right whale,Meredith Sackett,None,None,2023-11-29 00:00:00,2023-11-29 00:00:00,2024-11-29 00:00:00,nan,nan,Entrance to Chesapeake Bay,nan,In progress,GeoTiff,Radiometrically Corrected (1B),https://cidr.cr.usgs.gov/request/view/6826/,Imagery will be shared with GAIA project colla...
1,2,6827,70,nan,North Atlantic right whale,Meredith Sackett,None,None,2023-11-29 00:00:00,2023-11-29 00:00:00,2024-11-29 00:00:00,nan,nan,Entrance to New York Harbor,nan,In progress,GeoTiff,Radiometrically Corrected (1B),nan,nan
2,3,6828,71,nan,North Atlantic right whale,Meredith Sackett,None,None,2023-11-29 00:00:00,2023-11-29 00:00:00,2024-11-29 00:00:00,nan,nan,George's Basin / Fundian Channel,nan,In progress,GeoTiff,Radiometrically Corrected (1B),nan,nan
3,4,6829,72,nan,North Atlantic right whale,Meredith Sackett,None,None,2023-11-29 00:00:00,2023-11-29 00:00:00,2024-11-29 00:00:00,nan,nan,Maine Acoustic Target,nan,In progress,GeoTiff,Radiometrically Corrected (1B),nan,nan
4,5,6666,68,nan,North Atlantic right whale,Tyler Aldrich,None,None,2023-09-21 00:00:00,2023-09-21 00:00:00,2024-09-21 00:00:00,15 degrees,40 degrees,SE Nantucket Real time Buoy Target - Station 4...,submitted for 21 days but mentioned in the com...,nan,GeoTiff,Radiometrically Corrected (1B),https://cidr.cr.usgs.gov/request/view/6666/,nan


# End